# Premissa

Este desafio visa aplicar os modelos construídos anteriormente em um novo mapa com a finalidade de encontrar um caminho mais curto.

## Desvendando a Problemática

### Mapa do Desafio
![alternative text](./mapa_desafio.png)

### Distância em Linha Reta
![alternative text](./distancia_linha_reta.png)

1) Estado inicial: cidade de Porto União;
2) Estado final: cidade de Curitiba;
3) Espaço de estados: todas as possíveis caminhos entre Porto União e Curitiba, ex.:
> * Porto União >> São Mateus >> Lapa >> Contenda >> Araucária >> Curitiba;
> * Porto União >> Canoinhas >> Mafra >> Tijucas do Sul >> São José dos Pinhais >> Curitiba;
> * Porto União >> Paulo Frontin >> Irati >> Palmeira >> Campo Largo >> Balsa Nova >> Curitiba;
4) Ações de passagem: ir de uma cidade até outra, ex.:
> * Porto União >> São Mateus;
> * Palmeira >> Campo Largo;
> * Lapa >> Contenda;
5) Solução: processamento dos caminhos buscando encontrar a distância mais curta;


# Imports

In [62]:
import json
import numpy as np
from module import Mapa
from typing import List

# Vértice

Para o desenvolvimento deste desafio, será mantido a classe de Vértice construída nos arquivos anteriores;

In [2]:
class Vertice:
    '''
    Classe responsável por criar as cidades
    '''
    def __init__(self, rotulo, distancia_objetivo):
        self.rotulo = rotulo
        self.distancia_objetivo = distancia_objetivo
        self.visitado = False
        self.adjacentes = []

    def adiciona_adjacentes(self, adjacente):
        self.adjacentes.append(adjacente)

    def mostra_adjacentes(self):
        for i in self.adjacentes:
            print(f'{i.vertice.rotulo}: {i.custo}')

# Adjacente

O mesmo se aplica a classe adjacente que será reaproveitada nos arquivos anteriores, no caso desta classe optou-se por fazer uso da versão implementada no arquivo aestrela, visto que esta é a versão mais completa

In [19]:
class Adjacente:
    '''
    Classe responsável por fazer a conexão entre as cidades
    '''
    def __init__(self, vertice, custo):
        self.vertice = vertice
        self.custo = custo
        self.distancia_aestrela = self.vertice.distancia_objetivo + self.custo

# Vetor Ordenado

Por fim o Vetor Ordenado também será reutilizado por estar em bom estado de reaproveitamento.

In [ ]:
class VetorOrdenado:
    '''
    Classe responsável por criar um vetor de ordenação de valores focado na distancia_aestrela das adjacentes
    '''
    def __init__(self, capacidade):
        self.capacidade = capacidade
        self.ultima_posicao = -1
        self.lista_adjacentes = np.empty(self.capacidade, dtype=object)

    def mostrar_vetor(self):
        if self.ultima_posicao == -1:
            print('Vetor Vazio')
        else:
            for i in range(self.ultima_posicao + 1):
                print('{} - {} - {} - {} - {}'.format(
                    i,
                    self.lista_adjacentes[i].vertice.rotulo,
                    self.lista_adjacentes[i].custo,
                    self.lista_adjacentes[i].vertice.distancia_objetivo,
                    self.lista_adjacentes[i].distancia_aestrela
                ))

    def adicionar_valor_vetor(self, adjacente):
        if self.ultima_posicao == self.capacidade - 1:
            print('Capacidade máxima alcançada.')
        else:
            posicao = 0
            for i in range(self.ultima_posicao + 1):
                posicao = i
                if self.lista_adjacentes[i].distancia_aestrela > adjacente.distancia_aestrela:
                    break
                if i == self.ultima_posicao:
                    posicao = i + 1

            x = self.ultima_posicao
            while x >= posicao:
                self.lista_adjacentes[x + 1] = self.lista_adjacentes[x]
                x -= 1

            self.lista_adjacentes[posicao] = adjacente
            self.ultima_posicao += 1

# grafo

Para a classe da cosntrução do Grafo, será implementado um modelo de criação automática que irá ler um arquivo json e converter ele no Grafo do desagio.

## Desenvolvimento de automação para criação de Grafo

In [ ]:
# Chamando json do mapa
with open('mapa.json', encoding='utf-8') as file:
    mapa: List[Mapa] = json.load(file)
print(mapa)

In [63]:
# Definindo Classes construtora
def adicionar_adjacente_grafo(cidade, vertice: Vertice, grafo: List[Vertice]):
        vertice.adiciona_adjacentes(
            Adjacente(
                next(x for x in grafo if x.rotulo == cidade['nome']),
                cidade['distancia_real']
            )
        )
        return vertice

def encontrar_cidade_adjacente(grafo: List[Vertice]):
    for vertice in grafo:
        cidade = next(x for x in mapa if x['nome'] == vertice.rotulo)
        for adjacente in cidade['adjacentes']:
            vertice = adicionar_adjacente_grafo(
                adjacente, 
                vertice, 
                grafo
            )
    return grafo

def criar_vertices_grafo():
    grafo: List[Vertice] = []
    for cidade in mapa:
        vertice = Vertice(cidade['nome'], cidade['distancia_linha_reta'])
        grafo.append(vertice)
    return encontrar_cidade_adjacente(grafo)

In [72]:
#Testando funcionamento das classe de criação de grafo
grafo = criar_vertices_grafo()
for item in grafo:
    print('--------------------------------------------------')
    print(f'Cidade: {item.rotulo}')
    print(f'Distância Linha Reta: {item.distancia_objetivo}')
    print('Adjacentes: | Dist Real | Dist Linha Reta | Soma |')
    for cidade in item.adjacentes:
        print('    {} - {} - {} - {}'.format(
            cidade.vertice.rotulo,
            cidade.custo,
            cidade.vertice.distancia_objetivo,
            cidade.distancia_aestrela
        ))

--------------------------------------------------
Cidade: Porto União
Distância Linha Reta: 203
Adjacentes: | Dist Real | Dist Linha Reta | Soma |
    Paulo Frontin - 46 - 172 - 218
    São Mateus - 87 - 123 - 210
    Canoinhas - 78 - 141 - 219
--------------------------------------------------
Cidade: Paulo Frontin
Distância Linha Reta: 172
Adjacentes: | Dist Real | Dist Linha Reta | Soma |
    Irati - 75 - 139 - 214
    Porto União - 46 - 203 - 249
--------------------------------------------------
Cidade: Canoinhas
Distância Linha Reta: 141
Adjacentes: | Dist Real | Dist Linha Reta | Soma |
    Porto União - 78 - 203 - 281
    Três Barras - 12 - 131 - 143
    Mafra - 66 - 94 - 160
--------------------------------------------------
Cidade: Três Barras
Distância Linha Reta: 131
Adjacentes: | Dist Real | Dist Linha Reta | Soma |
    São Mateus - 43 - 123 - 166
    Canoinhas - 12 - 141 - 153
--------------------------------------------------
Cidade: São Mateus
Distância Linha Reta: 123